## Historical NBA Playoff Brackets

This notebook demonstrates how to build a data set of historical NBA playoff match ups. We will combine box scores from [stats.nba.com](http://stats.nba.com/) with historical playoff information from [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/). I picked this site because it has HTML tables which are easy to scrape.

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:.3f}'.format

In [2]:
import collections

In [3]:
from tqdm import tqdm_notebook as tqdm

We will use the [`pracnbastats`](https://pypi.org/project/pracnbastats/) package to scrape [stats.nba.com](http://stats.nba.com/). You can install this package in your sports analytics Python environment by executing `pip install pracnbastats` in Terminal (on Mac or Linux computers) or at the Anaconda Prompt (on Windows computers).

In [4]:
import pracnbastats as nba

We will use the HTML table scraping function from the [`pracpred`](https://pypi.org/project/pracpred/) package to scrape [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/). You can install this package in your sports analytics Python environment by executing `pip install pracpred` at the command line.

In [5]:
import pracpred.scrape as pps

In [6]:
from pathlib import Path

This code assumes the existence of a directory to hold scraped NBA data. You can create and name this directory however you want, and adjust the code in the cell below to suit your preferences. If you've previously scraped the data, the `pracnbastats` library can find it and avoid re-scraping. You just need to specify the location of the previously scraped data using the `store` object defined below.

In [7]:
PROJECT_DIR = Path.cwd().parent
DATA_DIR = PROJECT_DIR / 'data'
STATS_DIR = DATA_DIR / 'stats-nba-com'
OUTPUT_DIR = DATA_DIR / 'prepared'

To scrape data from [stats.nba.com](http://stats.nba.com/), you need to specify a user agent. Below is the user agent I used. You can find your own user agent by searching for "my user agent" in Google.

In [8]:
USER_AGENT = (
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) '
    'AppleWebKit/537.36 (KHTML, like Gecko) '
    'Chrome/66.0.3359.139 Safari/537.36'
)

In [9]:
session = nba.scrape.NBASession(user_agent=USER_AGENT)

In [10]:
store = nba.store.FlatFiles.CSV(path=STATS_DIR)

In [11]:
scraper = nba.scrape.NBAScraper(session=session, store=store)

### Historical Team Information

As we've [discussed in previous posts](http://practicallypredictable.com/2017/12/21/web-scraping-nba-team-matchups-box-scores/#hornets-and-pelicans), we need to keep track of franchise moves and name changes to look at team performance consistently over time. In particular, the Charlotte/New Orleans franchise history is a bit confused in the historical NBA data. The `teamhistory` module in the `pracnbastats` package contains some useful functions to consistently analyze historical team information.

In [12]:
nba.teamhistory.load(scraper=scraper, fix_hornets=True)

We will also need to look up current team information and link it with the historical team information.

In [13]:
nba.currentteams.load(scraper=scraper)

### Historical Playoff Match Ups

We will use the concept of a match up ID to group games between two teams in a particular season. We introduced the match up ID in our [earlier notebook](https://nbviewer.jupyter.org/github/practicallypredictable/posts/blob/master/basketball/nba/notebooks/nba-playoffs-home_court_advantage.ipynb) examining NBA playoff home court advantage.

In [14]:
def matchup_id(season_year, abbrs):
    """Unique identifier for a matchup between two team abbreviations in a particular season"""
    assert len(abbrs) == 2
    abbrs = sorted(list(abbrs))
    abbr1 = abbrs.pop(0)
    abbr2 = abbrs.pop(0)
    return f'{abbr1}_{abbr2}_{season_year}'

In [15]:
def create_matchup_ids(row):
    """Create matchup identifier for a DataFrame row"""
    return matchup_id(row['season'], set([row['team_abbr_h'], row['team_abbr_r']]))

Let's get the historical playoff box scores from the 1996-97 through 2016-17 NBA seasons, and add a column for the match up ID applicable to each game.

In [16]:
def load_season_matchups(
        *, scraper,
        season_type=nba.params.SeasonType.Regular,
        start_season=nba.params.MIN_YEAR,
        end_season=nba.params.Season.current_start_year()):
    """NBA match ups from stats.nba.com"""
    seasons = {
        season: nba.team.BoxScores(
            scraper=scraper,
            season=nba.params.Season(start_year=season),
            season_type=season_type,
        ) for season in tqdm(range(start_season, end_season+1))
    }
    df = pd.concat(seasons[season].matchups for season in seasons)
    df['matchup_id'] = df.apply(create_matchup_ids, axis=1)
    first_cols = ['matchup_id', ]
    cols = first_cols + [col for col in df.columns if col not in first_cols]
    return df[cols].sort_values(by=['date']).reset_index(drop=True)

In [17]:
nba_post = load_season_matchups(
    scraper=scraper,
    season_type=nba.params.SeasonType.Playoffs,
    end_season=nba.params.Season.current_start_year()-1)

In [18]:
len(nba_post)

1686

### Summarizing Playoff Series Outcomes

Now we are going to create a table with one row for each historical playoff match up. Our goal is to use this table to analyze how well a ratings system (such as our simple Elo ratings system) would have predicted the outcome of the playoff series, given the information available at the time. Each row in this table should contain information on the teams involved in that particular match up, which team had series home court advantage, which games were played at home for which teams, and which games were won by which team.

We will use the `pandas` [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) functionality to aggregate the game data by match up ID. The functions below referencing a group will be called within a `groupby()` aggregation for each match up ID.

#### Season and Date of First Game

First, we need to get the season year for each group.

In [19]:
def season_for_group(g):
    return g['season'].min()  # all rows have same season

We will also keep track of the first game date for each series, so we can arrange playoff series in chronological order.

In [20]:
def first_game_date_for_group(g):
    return g['date'].min()

#### Game Location

The next few functions will determine where each game in the series was played. Let's work through them one step at a time.

The function below takes a string of 3-letter team abbreviations (without punctuation) and breaks it into a list of separate abbreviations.

In [21]:
def split_abbrs(s):
    return [s[i:i+3] for i in range(0, len(s), 3)]

The purpose of the next function is to create a list of abbreviations for which team played which game at home. The `sum()` method will take each home team abbreviation and concatenate it into a long string. We can then use the above function to split this

In [22]:
def home_team_abbrs_for_group(g):
    return split_abbrs(g['team_abbr_h'].sum())

The next function in this set turns the list of team abbreviations into a string of yes (`'Y'`) and no (`'N'`) symbols. A `Y` character means that the team with series home court advantage played that game at home. The character location within the string corresponds to the number of the game within the series.

So, for example, a best-of-7 second round series with 5 games played would have the string `'YYNNY'`. This means that the first two games and the fifth game were played on the home court of the team with series home court advantage.

In [23]:
def hca_team_at_home(games, hca_abbr, non_hca_abbr):
    games = [s.replace(hca_abbr, 'Y') for s in games]
    games = [s.replace(non_hca_abbr, 'N') for s in games]
    return ''.join(games)

#### Game Winner

The next two functions do something very similar, but tell us which team won which games in the playoff series. In these functions, `'Y'` means that the team with series home court advantage won the game, while `'N'` means that team lost the game.

In [24]:
def winner_abbrs_for_group(g):
    return split_abbrs(g['winner'].sum())

In [25]:
def hca_won(games, hca_abbr, non_hca_abbr):
    games = [s.replace(hca_abbr, 'Y') for s in games]
    games = [s.replace(non_hca_abbr, 'N') for s in games]
    return ''.join(games)

Now we can put everything together in one function that will be called for each match up ID. This is a long function, but conceptually it is very simple. The argument it receives is a group containing all of  the games for a match up ID. It calls the above aggregation functions for this group. Then, it fills in various team information. The function returns a `pandas` [`Series`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) object containing the data for the group. Each of these groups will eventually become a row in a `DataFrame`.

In [26]:
def playoff_matchup(g):
    """Summarize playoff series result for a given match up ID"""
    record = {}
    # Get the season and date of first game in series
    record['season'] = season_for_group(g)
    record['first_game_date'] = first_game_date_for_group(g)
    # Get the team abbreviations
    home_team_abbrs = home_team_abbrs_for_group(g)
    # Get information for team with series home court advantage
    # The team with series home court advantage plays at home first
    record['hca_team_abbr'] = home_team_abbrs[0]
    record['hca_team_id'] = nba.teamhistory.id_for_abbr(record['hca_team_abbr'])
    hca = nba.teamhistory.select(season=record['season'], team_id=record['hca_team_id'])
    record['hca_team_city'] = hca.city
    record['hca_team_name'] = hca.name
    record['hca_team_reg_wins'] = hca.wins
    record['hca_team_reg_losses'] = hca.losses
    record['hca_team_conf_rank'] = hca.conf_rank
    record['hca_team_div_rank'] = hca.div_rank
    # Get the abbreviation for the other team (without series home court advantage)
    other_teams = set([team for team in home_team_abbrs if team != record['hca_team_abbr']])
    assert len(other_teams) == 1
    # Get information for the other team
    record['non_hca_team_abbr'] = other_teams.pop()
    record['non_hca_team_id'] = nba.teamhistory.id_for_abbr(record['non_hca_team_abbr'])
    non_hca = nba.teamhistory.select(season=record['season'], team_id=record['non_hca_team_id'])
    record['non_hca_team_city'] = non_hca.city
    record['non_hca_team_name'] = non_hca.name
    record['non_hca_team_reg_wins'] = non_hca.wins
    record['non_hca_team_reg_losses'] = non_hca.losses
    record['non_hca_team_conf_rank'] = non_hca.conf_rank
    record['non_hca_team_div_rank'] = non_hca.div_rank
    winner_abbrs = winner_abbrs_for_group(g)  # which team won which games
    # Count how many games each team won
    games_won = collections.Counter(winner_abbrs)
    record['hca_series_wins'] = (
        games_won[record['hca_team_abbr']]
        if record['hca_team_abbr'] in games_won else 0
    )
    record['non_hca_series_wins'] = (
        games_won[record['non_hca_team_abbr']]
        if record['non_hca_team_abbr'] in games_won else 0
    )
    # Record series best-of and total number of games played
    record['best_of'] = 7 if max(record['hca_series_wins'], record['non_hca_series_wins']) == 4 else 5
    record['games'] = record['hca_series_wins'] + record['non_hca_series_wins']
    # Record series winner
    record['series_winner'] = (
        record['hca_team_abbr']
        if record['hca_series_wins'] > record['non_hca_series_wins']
        else record['non_hca_team_abbr']
    )
    # Record where each game was played
    record['hca_at_home'] = hca_team_at_home(
        home_team_abbrs,
        record['hca_team_abbr'],
        record['non_hca_team_abbr'],
    )
    # Record which team won each game
    record['hca_won'] = hca_won(
        winner_abbrs,
        record['hca_team_abbr'],
        record['non_hca_team_abbr'],
    )
    # Reorder columns
    cols = [
        'season',
        'hca_team_abbr',
        'non_hca_team_abbr',
        'hca_at_home',
        'hca_won',
        'best_of',
        'hca_series_wins',
        'non_hca_series_wins',
        'games',
        'series_winner',
        'hca_team_id',
        'hca_team_city',
        'hca_team_name',
        'hca_team_reg_wins',
        'hca_team_reg_losses',
        'hca_team_conf_rank',
        'hca_team_div_rank',
        'non_hca_team_id',
        'non_hca_team_city',
        'non_hca_team_name',
        'non_hca_team_reg_wins',
        'non_hca_team_reg_losses',
        'non_hca_team_conf_rank',
        'non_hca_team_div_rank',
        'first_game_date',
    ]
    return pd.Series(record, index=cols)

All we need to do now is loop through all the playoff match up IDs to create our new table.

In [27]:
def playoff_matchups(matchups):
    df = (
        matchups.groupby(['matchup_id'])
        .apply(playoff_matchup)
        .reset_index()
        .sort_values(by=['first_game_date'])
        .reset_index(drop=True)
    )
    return df

In [28]:
df = playoff_matchups(nba_post)
df.head()

,matchup_id,season,hca_team_abbr,non_hca_team_abbr,hca_at_home,hca_won,best_of,hca_series_wins,non_hca_series_wins,games,series_winner,hca_team_id,hca_team_city,hca_team_name,hca_team_reg_wins,hca_team_reg_losses,hca_team_conf_rank,hca_team_div_rank,non_hca_team_id,non_hca_team_city,non_hca_team_name,non_hca_team_reg_wins,non_hca_team_reg_losses,non_hca_team_conf_rank,non_hca_team_div_rank,first_game_date
0,HOU_MIN_1996,1996,HOU,MIN,YYN,YYY,5,3,0,3,HOU,1610612745,Houston,Rockets,57,25,2,2,1610612750,Minnesota,Timberwolves,40,42,6,3,1997-04-24
1,MIA_ORL_1996,1996,MIA,ORL,YYNNY,YYNNY,5,3,2,5,MIA,1610612748,Miami,Heat,61,21,2,1,1610612753,Orlando,Magic,45,37,7,3,1997-04-24
2,CHH_NYK_1996,1996,NYK,CHH,YYN,YYY,5,3,0,3,NYK,1610612752,New York,Knicks,57,25,3,2,1610612740,Charlotte,Hornets,54,28,5,3,1997-04-24
3,LAC_UTA_1996,1996,UTA,LAC,YYN,YYY,5,3,0,3,UTA,1610612762,Utah,Jazz,64,18,1,1,1610612746,Los Angeles,Clippers,36,46,8,5,1997-04-24
4,PHX_SEA_1996,1996,SEA,PHX,YYNNY,NYNYY,5,3,2,5,SEA,1610612760,Seattle,SuperSonics,57,25,2,1,1610612756,Phoenix,Suns,40,42,6,4,1997-04-25


### Scraping Historical NBA Playoff Brackets

Now we are going to switch gears and scrape the historical playoff bracket information from [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/). We can use this site to classify match ups by playoff round, conference or seed.

#### Team Names

First, we will generate a list of the various historical team full names (city plus franchise name) from our existing data. We will see that some of the team names used on [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/) differ from [stats.nba.com](http://stats.nba.com/). We will need to do a few adjustments to get the team names to line up correctly.

In [29]:
def unique_nba_teams(df):
    return (
        set(df['hca_team_city'].str.cat(df['hca_team_name'], sep=' ').tolist())
        .union(set(df['non_hca_team_city'].str.cat(df['non_hca_team_name'], sep=' ').tolist()))
    )

In [30]:
nba_teams = unique_nba_teams(df)
len(nba_teams)

36

Now we are ready to do some scraping. The procedure is very simple and laid out in the next few functions:

1. Loop through all NBA seasons from 1996-97 through 2016-17.
2. Each playoff series in a particular season is represented as a small HTML table of a given class.
3. Process each of the HTML tables and create a record of each playoff series information.

In [31]:
def parse_matchup(df):
    """Real GM record of particular playoff series"""
    assert len(df) == 2
    rows = df.to_dict(orient='records')
    upper = rows[0]
    lower = rows[1]
    assert upper['playoff_round'] == lower['playoff_round']
    best_of = 7 if max(upper['games_won'], lower['games_won']) == 4 else 5
    return collections.OrderedDict(
        playoff_round=upper['playoff_round'],
        best_of=best_of,
        upper_conference=upper['conference'],
        upper_seed=upper['seed'],
        upper_team=upper['team'],
        upper_won=upper['games_won'],
        lower_conference=lower['conference'],
        lower_seed=lower['seed'],
        lower_team=lower['team'],
        lower_won=lower['games_won'],
    )

In [32]:
def format_bracket(df, game_index):
    """Format raw Real GM playoff bracket data"""
    df = df.rename(columns={
        0: 'seed',
        1: 'team',
        2: 'games_won',
    })
    df['seed_code'] = df['seed']
    df['conference'] = df['seed'].str[0]
    df['seed'] = df['seed'].str[1].astype(int)
    df['games_won'] = df['games_won'].astype(int)
    if game_index <= 7:
        playoff_round = 1 # conference quarter-finals
    elif game_index <= 11:
        playoff_round = 2 # conference semi-finals
    elif game_index <= 13:
        playoff_round = 3 # conference finals
    elif game_index == 14:
        playoff_round = 4 # finals
    else:
        raise RuntimeError('unknown error')
    df['playoff_round'] = playoff_round
    return parse_matchup(df)

In order to compute series win probabilities, we need to know where every possible game in the series would have been played, if the series went the distance. The function below determines the same sort  of yes/no string, assuming the series went to a decisive final game.

The function below handles the various changes in playoff format since 1996. For example, prior to the 2002-3 season, first round NBA playoff games were a best-of-5 format. The finals format also changed during this period. We need to know the playoff round in order to handle these playoff format changes. Fortunatley, the [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/) information provides the playoff round.

In [33]:
def hca_home_schedule(season, playoff_round):
    assert season > 1984  # Beginning of modern playoff format
    if season < 2002 and playoff_round == 1:
        return 'YYNNY'  # Old best-of-5 first round
    elif playoff_round == 4:  # NBA finals
        if season < 2013:
            return 'YYNNNYY'  # Old 2-3-2 finals format
        else:
            return 'YYNNYNY'  # Current 2-2-1-1-1 finals format
    else:
        return 'YYNNYNY'  # All other rounds best-of-7 playoff round format

Now we can process the tables.

In [34]:
def tables_to_df(tables, season):
    """Process all Real GM playoff series for a particular season"""
    rows = []
    for i, table in enumerate(tables):
        df = table.to_df()
        row = format_bracket(df, game_index=i)
        row['season'] = season
        row['hca_home_schedule'] = hca_home_schedule(
            season=season,
            playoff_round=row['playoff_round'],
        )
        rows.append(row)
    df = pd.DataFrame.from_records(rows)
    int_cols = [
        'season',
        'best_of',
        'upper_seed',
        'upper_won',
        'lower_seed',
        'lower_won',
    ]
    for col in int_cols:
        df[col] = df[col].astype(int)
    return df

In [35]:
def scrape_brackets(
        start_year=nba.params.MIN_YEAR,
        end_year=nba.params.Season.current_start_year()-1):
    """Scrape Real GM site for historical playoff brackets"""
    BASE_URL = 'https://basketball.realgm.com/nba/playoffs/brackets/'
    TABLE_CLASS = 'bracket_game'
    tables = {
        season: pps.HTMLTables(url=f'{BASE_URL}{season+1}', table_class=TABLE_CLASS)
        for season in tqdm(range(start_year, end_year+1))
    }
    df = pd.concat(tables_to_df(tables[season], season) for season in tables).reset_index(drop=True)
    first_cols = ['season', 'playoff_round', 'best_of', 'hca_home_schedule']
    cols = first_cols + [col for col in df.columns if col not in first_cols]
    return df[cols]

In [36]:
brackets = scrape_brackets()
len(brackets)

315

In [37]:
brackets.head()

,season,playoff_round,best_of,hca_home_schedule,upper_conference,upper_seed,upper_team,upper_won,lower_conference,lower_seed,lower_team,lower_won
0,1996,1,5,YYNNY,E,1,Chicago Bulls,3,E,8,Washington Bullets,0
1,1996,1,5,YYNNY,E,4,Atlanta Hawks,3,E,5,Detroit Pistons,2
2,1996,1,5,YYNNY,E,3,New York Knicks,3,E,6,Charlotte Hornets,0
3,1996,1,5,YYNNY,E,2,Miami Heat,3,E,7,Orlando Magic,2
4,1996,1,5,YYNNY,W,1,Utah Jazz,3,W,8,Los Angeles Clippers,0


### Merging the Historical Information

Now we need to figure out how to merge the two data sets. We want to merge on season and team name. In order to do that, we have to make sure that the team names match. Let's check.

In [38]:
def unique_bracket_teams(df):
    return (
        set(df['upper_team'].unique().tolist())
        .union(set(df['lower_team'].unique().tolist()))
    )

In [39]:
bracket_teams = unique_bracket_teams(brackets)
len(bracket_teams)

35

These are team names that appear in the [stats.nba.com](http://stats.nba.com/) data but not in the [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/) data.

In [40]:
nba_teams - bracket_teams

{'LA Clippers', 'Philadelphia 76ers'}

Let's look at what's going on with the Clippers.

In [41]:
df[(df['hca_team_abbr'] == 'LAC') | (df['non_hca_team_abbr'] == 'LAC')]

,matchup_id,season,hca_team_abbr,non_hca_team_abbr,hca_at_home,hca_won,best_of,hca_series_wins,non_hca_series_wins,games,series_winner,hca_team_id,hca_team_city,hca_team_name,hca_team_reg_wins,hca_team_reg_losses,hca_team_conf_rank,hca_team_div_rank,non_hca_team_id,non_hca_team_city,non_hca_team_name,non_hca_team_reg_wins,non_hca_team_reg_losses,non_hca_team_conf_rank,non_hca_team_div_rank,first_game_date
3,LAC_UTA_1996,1996,UTA,LAC,YYN,YYY,5,3,0,3,UTA,1610612762,Utah,Jazz,64,18,1,1,1610612746,Los Angeles,Clippers,36,46,8,5,1997-04-24
137,DEN_LAC_2005,2005,LAC,DEN,YYNNY,YYNYY,7,4,1,5,LAC,1610612746,Los Angeles,Clippers,47,35,6,2,1610612743,Denver,Nuggets,44,38,4,1,2006-04-22
146,LAC_PHX_2005,2005,PHX,LAC,YYNNYNY,YNYNYNY,7,4,3,7,PHX,1610612756,Phoenix,Suns,54,28,3,1,1610612746,Los Angeles,Clippers,47,35,6,2,2006-05-08
231,LAC_MEM_2011,2011,MEM,LAC,YYNNYNY,NYNNYYN,7,3,4,7,LAC,1610612763,Memphis,Grizzlies,41,25,4,2,1610612746,Los Angeles,Clippers,40,26,5,2,2012-04-29
236,LAC_SAS_2011,2011,SAS,LAC,YYNN,YYYY,7,4,0,4,SAS,1610612759,San Antonio,Spurs,50,16,1,1,1610612746,Los Angeles,Clippers,40,26,5,2,2012-05-15
241,LAC_MEM_2012,2012,LAC,MEM,YYNNYN,YYNNNN,7,2,4,6,MEM,1610612746,Los Angeles,Clippers,56,26,4,1,1610612763,Memphis,Grizzlies,56,26,5,2,2013-04-20
257,GSW_LAC_2013,2013,LAC,GSW,YYNNYNY,NYYNYNY,7,4,3,7,LAC,1610612746,Los Angeles,Clippers,57,25,3,1,1610612744,Golden State,Warriors,51,31,6,2,2014-04-19
264,LAC_OKC_2013,2013,OKC,LAC,YYNNYN,NYYNYY,7,4,2,6,OKC,1610612760,Oklahoma City,Thunder,59,23,2,1,1610612746,Los Angeles,Clippers,57,25,3,1,2014-05-05
276,LAC_SAS_2014,2014,LAC,SAS,YYNNYNY,YNNYNYY,7,4,3,7,LAC,1610612746,Los Angeles,Clippers,56,26,3,2,1610612759,San Antonio,Spurs,55,27,6,3,2015-04-19
281,HOU_LAC_2014,2014,HOU,LAC,YYNNYNY,NYNNYYY,7,4,3,7,HOU,1610612745,Houston,Rockets,56,26,2,1,1610612746,Los Angeles,Clippers,56,26,3,2,2015-05-04


The problem here is that the Clippers formally changed their name to the LA Clippers in 2015, but [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/) is still calling them the Los Angeles Clippers. We'll need to override that.

Now let's look at team names that are in the [Real GM](https://basketball.realgm.com/nba/playoffs/brackets/) data, which don't appear in [stats.nba.com](http://stats.nba.com/).

In [42]:
bracket_teams - nba_teams

{'Philadelphia Sixers'}

This problem is relatively simple. We just need to change 'Sixers' to '76ers'. Let's put these fixes into a function.

In [43]:
def fix_bracket_teams(df):
    df['upper_team'].replace(to_replace='Philadelphia Sixers', value='Philadelphia 76ers', inplace=True)
    df['lower_team'].replace(to_replace='Philadelphia Sixers', value='Philadelphia 76ers', inplace=True)
    la_clippers = df[(df['season'] >= 2015) & (df['upper_team'] == 'Los Angeles Clippers')].index
    df.loc[la_clippers, 'upper_team'] = 'LA Clippers'
    la_clippers = df[(df['season'] >= 2015) & (df['lower_team'] == 'Los Angeles Clippers')].index
    df.loc[la_clippers, 'lower_team'] = 'LA Clippers'
    return df

In [44]:
brackets = fix_bracket_teams(brackets)

Now let's check if we have any name mismatches.

In [45]:
unique_nba_teams(df) - unique_bracket_teams(brackets)

set()

In [46]:
unique_bracket_teams(brackets) - unique_nba_teams(df)

set()

Success! Now we can merge the data sets.

In [47]:
def get_bracket_info(df, season, hca_team, non_hca_team):
    row = df[(df['season'] == season) & (df['upper_team'] == hca_team) & (df['lower_team'] == non_hca_team)]
    assert len(row) <= 1
    if len(row) == 1:
        return row.to_dict(orient='records')[0]
    else:
        row = df[(df['season'] == season) & (df['upper_team'] == non_hca_team) & (df['lower_team'] == hca_team)]
        assert len(row) == 1
        return row.to_dict(orient='records')[0]

Just to put everything in one place, the function below builds the entire merged data set from scratch. Having all the logic in one place makes it less likely we will create errors in our code or data down the road.

In [48]:
def playoff_brackets(matchups):
    games = playoff_matchups(matchups)
    start_year = games['season'].min()
    end_year = games['season'].max()
    brackets = scrape_brackets(start_year=start_year, end_year=end_year)
    brackets = fix_bracket_teams(brackets)
    records = []
    for row in games.itertuples(index=False, name='row'):
        record = row._asdict()
        hca_team = f'{row.hca_team_city} {row.hca_team_name}'
        non_hca_team = f'{row.non_hca_team_city} {row.non_hca_team_name}'
        bracket_info = get_bracket_info(brackets, season=row.season, hca_team=hca_team, non_hca_team=non_hca_team)
        record['playoff_round'] = bracket_info['playoff_round']
        record['hca_home_schedule'] = bracket_info['hca_home_schedule']
        if bracket_info['upper_team'] == hca_team:
            record['hca_seed'] = bracket_info['upper_seed']
            record['hca_conference'] = 'East' if bracket_info['upper_conference'] == 'E' else 'West'
            record['non_hca_seed'] = bracket_info['lower_seed']
            record['non_hca_conference'] = 'East' if bracket_info['lower_conference'] == 'E' else 'West'
        else:
            record['hca_seed'] = bracket_info['lower_seed']
            record['hca_conference'] = 'East' if bracket_info['lower_conference'] == 'E' else 'West'
            record['non_hca_seed'] = bracket_info['upper_seed']
            record['non_hca_conference'] = 'East' if bracket_info['upper_conference'] == 'E' else 'West'
        records.append(record)
    df = pd.DataFrame.from_records(records)
    df = df.drop(columns=[
        'hca_team_city',
        'hca_team_name',
        'non_hca_team_city',
        'non_hca_team_name',
    ])
    first_cols = [
        'season',
        'playoff_round',
        'matchup_id',
        'series_winner',
        'hca_at_home',
        'hca_won',
        'games',
        'best_of',
        'hca_home_schedule',
        'hca_team_abbr',
        'hca_series_wins',
        'hca_conference',
        'hca_seed',
        'non_hca_team_abbr',
        'non_hca_series_wins',
        'non_hca_conference',
        'non_hca_seed',
    ]
    cols = first_cols + [col for col in df.columns if col not in first_cols]
    return df[cols]

In [49]:
df = playoff_brackets(nba_post)
len(df)

315

In [50]:
df.head()

,season,playoff_round,matchup_id,series_winner,hca_at_home,hca_won,games,best_of,hca_home_schedule,hca_team_abbr,hca_series_wins,hca_conference,hca_seed,non_hca_team_abbr,non_hca_series_wins,non_hca_conference,non_hca_seed,hca_team_id,hca_team_reg_wins,hca_team_reg_losses,hca_team_conf_rank,hca_team_div_rank,non_hca_team_id,non_hca_team_reg_wins,non_hca_team_reg_losses,non_hca_team_conf_rank,non_hca_team_div_rank,first_game_date
0,1996,1,HOU_MIN_1996,HOU,YYN,YYY,3,5,YYNNY,HOU,3,West,3,MIN,0,West,6,1610612745,57,25,2,2,1610612750,40,42,6,3,1997-04-24
1,1996,1,MIA_ORL_1996,MIA,YYNNY,YYNNY,5,5,YYNNY,MIA,3,East,2,ORL,2,East,7,1610612748,61,21,2,1,1610612753,45,37,7,3,1997-04-24
2,1996,1,CHH_NYK_1996,NYK,YYN,YYY,3,5,YYNNY,NYK,3,East,3,CHH,0,East,6,1610612752,57,25,3,2,1610612740,54,28,5,3,1997-04-24
3,1996,1,LAC_UTA_1996,UTA,YYN,YYY,3,5,YYNNY,UTA,3,West,1,LAC,0,West,8,1610612762,64,18,1,1,1610612746,36,46,8,5,1997-04-24
4,1996,1,PHX_SEA_1996,SEA,YYNNY,NYNYY,5,5,YYNNY,SEA,3,West,2,PHX,2,West,7,1610612760,57,25,2,1,1610612756,40,42,6,4,1997-04-25


Now that we have all the data cleaned up in one place, let's save it to a CSV file. We will use this table to analyze historical NBA playoff series in future posts.

In [51]:
csvfilename = 'nba-playoff_brackets-1996_2016.csv'
csvfile = OUTPUT_DIR.joinpath(csvfilename)
df.to_csv(csvfile, index=False)